In [ ]:
from glob import glob
import random
import math
import os
import cv2
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plot
from random import sample 

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import DBSCAN
from sklearn.neighbors import NearestNeighbors

In [ ]:
images_path = glob('mura_data/RGB/mura_march_clean/train_data/normal/*.png')

limit = 1000

print(len(images_path))

if len(images_path) < limit or limit == 0:
    print("The amount of dataset smaller than limit so we will use all images in dataset.")
else:
    images_path = sample(images_path,limit)
    
# def dbscan_preprocessing(images_path, limit=None):
    # images_path_array = glob(images_path)
print(len(images_path))   
df_analysis = pd.DataFrame(columns=['image_path','mean','std'])

for img_path in images_path:
    image = cv2.imread(img_path)
    # print(image)
    mean = np.mean(image)
    std = np.std(image)
    # print(mean, image.mean())
    # print(std, image.std())
    data_row = {
        "image_path": img_path,
        "mean": image.mean(),
        "std": image.std(),
        # "class": 0
    }
    df_analysis = df_analysis.append(data_row, ignore_index = True)

In [ ]:
final_df = df_analysis.sort_values(['std', 'mean'], ascending = [True, False])
# Draw a scatter plot
final_df.plot.scatter(x = 'mean', y = 'std', s = 3, c="red")

In [ ]:
final_df.head(1000).plot.scatter(x = 'mean', y = 'std', s = 3, c="red")

In [ ]:
final_image_path = final_df['image_path'].head(100).tolist()

In [ ]:
def selecting_images_preprocessing(images_path_array, limit_image_to_process="MAX", limit_image_to_train = "MAX"):
    # images_path_array = glob(images_path)
    
    original_number_number_image = len(images_path_array)
    
    print("original number of data: ", original_number_number_image)
    
    if limit_image_to_train == "MAX":
        limit_image_to_train = original_number_number_image
    
    
    if len(images_path) < limit_image_to_process:
        print("The amount of dataset smaller than limit so we will use all images in dataset.")
    elif limit_image_to_process == "MAX":
        print("You choose to use all of data. please wait it will take a moment.")
    else:
        images_path_array = sample(images_path_array,limit_image_to_process)
    print("processed number of data: ", len(images_path_array))
    
    df_analysis = pd.DataFrame(columns=['image_path','mean','std'])

    for img_path in images_path_array:
        image = cv2.imread(img_path)
        # print(image)
        mean = np.mean(image)
        std = np.std(image)
        # print(mean, image.mean())
        # print(std, image.std())
        data_row = {
            "image_path": img_path,
            "mean": image.mean(),
            "std": image.std(),
            # "class": 0
        }
        df_analysis = df_analysis.append(data_row, ignore_index = True)
    final_df = df_analysis.sort_values(['std', 'mean'], ascending = [True, False])
    
    final_image_path = final_df['image_path'].head(limit_image_to_train).tolist()
    return final_image_path